In [1]:
import pandas as pd
from datetime import datetime as dt

In [66]:
# 수집 종목 코드 interation
stock_code = "117930"

In [104]:
# 시세 수집
page_max = 2 + 1
page = 1
url_daily_sise = "http://finance.daum.net/item/quote_yyyymmdd.daum?code=" + stock_code + "&page="+ str(page)
df = pd.read_html(url_daily_sise, header=0)[0]
df.columns = ['date','open','high','low','close','updown','rate','volume']
df['date']  = pd.to_datetime('20'+df['date'])
df[['date','open','high','low','close','volume']]
#df.set_index('date', inplace=True)
df


,date,open,high,low,close,updown,rate,volume
0,2016-05-02,1945,2205,1925,2010,▲110,+5.79%,16191560
1,2016-04-29,1825,1915,1775,1900,▲65,+3.54%,5881044
2,2016-04-28,1910,1920,1835,1835,▼65,-3.42%,5598265
3,2016-04-27,1945,1965,1875,1900,-0,0.00%,6565460
4,2016-04-26,1850,2050,1765,1900,▲75,+4.11%,43224665
5,2016-04-25,2020,2035,1825,1825,↓780,-29.94%,17297624
6,2016-04-22,2675,2710,2580,2605,▼205,-7.30%,6368785
7,2016-04-21,3015,3075,2775,2810,▼220,-7.26%,5514948
8,2016-04-20,3315,3355,2985,3030,▼355,-10.49%,4402720
9,2016-04-19,3350,3400,3250,3385,▲40,+1.20%,1411201


In [129]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import requests
import datetime, time
import pandas as pd
from pandas import DataFrame
import io
# user define package import
import sys
sys.path.append("/home/ubuntu/workspace/favis")
from msgbot.favisbot import favisbot
import util.krx_util as util

today = datetime.datetime.now().strftime('%Y%m%d')
print(today)

stock_name = '이미지스'
isu_cd, isu_srt_cd = util.getIsinCode(stock_name)
stock_code = isu_srt_cd

start = datetime.datetime(2016, 1, 1).strftime('%Y%m%d')
end = datetime.datetime(2016, 12, 31).strftime('%Y%m%d')

# STEP 01: Generate OTP
gen_otp_url = 'http://marketdata.krx.co.kr/contents/COM/GenerateOTP.jspx'
gen_otp_data = {
    'name':'fileDown',
    'filetype':'xls',
    'url':'MKD/04/0402/04020100/mkd04020100t3_02',
    'isu_srt_cd':isu_srt_cd,
    'isu_cd':isu_cd,
    'fromdate':start,
    'todate':end
}
    
r = requests.post(gen_otp_url, gen_otp_data)
code = r.text

# STEP 02: download
down_url = 'http://file.krx.co.kr/download.jspx'
down_data = {
    'code': code,
}

r = requests.post(down_url, down_data).content

filename = isu_srt_cd + ".xls"
with open(filename, 'wb') as f:
    #f.write(r.content.decode('utf-8'))
    f.write(r)

#df = pd.read_csv(io.StringIO(r.decode("utf-8")), thousands=',', parse_dates=[0], usecols=[0, '종가','거래량(주)','시가','고가','저가'])
df = pd.read_excel(filename, thousands=',', usecols=[0, '종가','거래량(주)','시가','고가','저가', '시가총액(백만)','상장주식수(주)'])

df.columns = ['Date','Adj Close','Volume','Open','High','Low', 'marcap','amount']
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
 
df.head()

20160502


,Date,Adj Close,Volume,Open,High,Low,marcap,amount
0,2016-04-29,6920,69214,6850,6940,6770,53070,7669128
1,2016-04-28,6850,132287,7020,7080,6850,52534,7669128
2,2016-04-27,7030,100859,7100,7200,6970,53914,7669128
3,2016-04-26,7130,124746,7200,7270,7030,54681,7669128
4,2016-04-25,7160,90460,7140,7220,7040,54911,7669128


In [130]:
# 매매 동향 수집
page_max = 3 + 1
page = 2

url_daily_trading_trends = "http://finance.daum.net/item/foreign_yyyymmdd.daum?code=" + stock_code + "&page=1"
print(url_daily_trading_trends)
df1 = pd.read_html(url_daily_trading_trends)[0]
#df = pd.read_html(daily_sise_url, index_col=[0], parse_dates=True)
df1.columns = ['date','fi_amount','fi_rate','fi_volume','ii_volume','close','ratebyyest''rate','nan','nan','NAN']
df1['date']  = pd.to_datetime('20'+df1['date'])
#df1.set_index('date', inplace=True)
df1 = df1[['date','fi_amount','fi_volume','ii_volume']]
df1['stock_code'] = stock_code
df_m = df1
for i in range(page, page_max):
    url_daily_trading_trends = "http://finance.daum.net/item/foreign_yyyymmdd.daum?code=" + stock_code + "&page="+ str(i)
    print(url_daily_trading_trends)
    df1 = pd.read_html(url_daily_trading_trends)[0]
    #df = pd.read_html(daily_sise_url, index_col=[0], parse_dates=True)
    df1.columns = ['date','fi_amount','fi_rate','fi_volume','ii_volume','close','ratebyyest''rate','nan','nan','NAN']
    df1['date']  = pd.to_datetime('20'+df1['date'])
    #df1.set_index('date', inplace=True)
    df1 = df1[['fi_amount','fi_volume','ii_volume']]
    df1['stock_code'] = stock_code
    df_m.append(df1)

result = pd.concat([df, df_m], axis=1)
result

http://finance.daum.net/item/foreign_yyyymmdd.daum?code=A115610&page=1
http://finance.daum.net/item/foreign_yyyymmdd.daum?code=A115610&page=2
http://finance.daum.net/item/foreign_yyyymmdd.daum?code=A115610&page=3


,Date,Adj Close,Volume,Open,High,Low,marcap,amount,date,fi_amount,fi_volume,ii_volume,stock_code
0,2016-04-29,6920,69214,6850,6940,6770,53070,7669128,2016-05-02,175231.0,-7361,0.0,A115610
1,2016-04-28,6850,132287,7020,7080,6850,52534,7669128,2016-04-29,182592.0,"+2,106",0.0,A115610
2,2016-04-27,7030,100859,7100,7200,6970,53914,7669128,2016-04-28,180486.0,"+7,131",0.0,A115610
3,2016-04-26,7130,124746,7200,7270,7030,54681,7669128,2016-04-27,173355.0,-952,0.0,A115610
4,2016-04-25,7160,90460,7140,7220,7040,54911,7669128,2016-04-26,174307.0,-2920,0.0,A115610
5,2016-04-22,7140,62004,7150,7150,7040,54758,7669128,2016-04-25,177227.0,"+3,797",0.0,A115610
6,2016-04-21,7120,165559,6890,7130,6860,54604,7669128,2016-04-22,173430.0,"+2,290",0.0,A115610
7,2016-04-20,6890,53591,6880,6940,6830,52840,7669128,2016-04-21,171140.0,"+13,138",0.0,A115610
8,2016-04-19,6830,70498,6830,6880,6710,52380,7669128,2016-04-20,158002.0,"+3,288",0.0,A115610
9,2016-04-18,6770,127532,6900,6900,6690,51920,7669128,2016-04-19,154714.0,"+4,694",0.0,A115610
